In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

%run ./lib.py


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# SIPP Inequality Extract
`census.gov-sipp-inequality-1.1.1` Last Update: 2021-01-02T00:42:01

_Select columns from SIPP for studies of income and wealth inequailty_


An extract of columns from the 2018 SIPP data, focused on income, poverty and correlates.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [sipp_18](notebooks/Extract.ipynb#df)**. SIPP Inequality extract, 2018

## References
<ul><li> <strong>SIPP</string>, <em>metapack+http://library.metatab.org/census.gov-sipp.csv</em>. SIPP Data package</li><ul>

In [64]:
# File cached in Extract.ipynb
# Cache the file for other notebooks. 
dfa = pd.read_feather('../data/inequality.feather')
dfa['dummy'] = 1
dfa['erel_cat'] = dfa.erelrpe.replace(erel_map)

df = dfa[dfa.monthcode==12]


In [5]:
# A lot of Americans live in households with other famlies; about 12% are not the primariy family in the household. 
# ( the primary familly may live with a second famly. )
t = df[df.monthcode == 12].groupby('rfamref').wpfinwgt.sum()
t/1e6

rfamref
101    283.407460
102     28.855232
103      4.891525
104      2.280309
105      0.936688
106      0.445650
107      0.293083
108      0.086486
109      0.034769
110      0.008164
111      0.003996
113      0.003524
Name: wpfinwgt, dtype: float64

In [6]:
# Each of the famlies that is not headed by person 101 lives with a family that does, so that 
# number of 101 famllies live with another family --> the factor of 2
t = df[df.monthcode == 12].groupby('rfamref').wpfinwgt.sum()
t = t.groupby(t.index==101).sum()/1e6

t_single = t.loc[True]-t.loc[False]
t_multiple = t.loc[False] * 2

# Number of people living in single and multiple family households
t_single, t_multiple, (t_multiple/t.sum())

(245.5680327977205, 75.67885520227752, 0.23557848505065668)

In [7]:
# Number of people in families in a household. ( How is there 1 person families?)
t =df.groupby(['ssuid','rfamref'])

t.dummy.count().value_counts().sort_index()

1     12885
2      8066
3      3593
4      2974
5      1330
6       534
7       182
8        68
9        36
10       13
11        4
12        1
13        1
14        2
Name: dummy, dtype: int64

In [8]:
# Number of people in households
t =df.groupby(['ssuid'])

t.dummy.count().value_counts().sort_index()

1     7908
2     8814
3     3772
4     3222
5     1508
6      595
7      235
8       87
9       45
10      13
11      10
12       1
13       3
14       2
Name: dummy, dtype: int64

In [56]:
cols = ['ssuid','wpfinwgt', 'rfamkind', 'rfpersons', 'rhnumper']

erelrpe_cols = ['erelrpe_'+e for e in erel_map.values()]

t = df[columns_by_group(['family_household','personal_finances','hh_finances'])]

t = t[t.monthcode==12][ cols +['erelrpe']].copy()

t['erelrpe'] = t.erelrpe.replace(erel_map)
t = pd.get_dummies(t, columns = ['erelrpe'])

# Reorder the erelrpe to be in the same order as the map. 
t = t[cols + erelrpe_cols]

agg_map = {
    'rfamkind': 'first',
    'rfpersons': 'first',
    'rhnumper': 'first'
}


for c in erelrpe_cols:
    agg_map[c] = 'sum'

x = t[t.rfpersons != t.rhnumper].groupby('ssuid').aggregate(agg_map)

x.T


ssuid,28503464618,28509223518,28533413818,28599519318,40618528318,40618594618,40628529418,40681629418,41803704018,41803713818,...,88199561077318,88199568830118,88199592528418,88199592916218,91022810168818,91092505716118,91092505768818,91092560676218,91092560950518,91092598831018
rfamkind,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0
rfpersons,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,4.0,2.0,1.0,4.0,1.0
rhnumper,2.0,4.0,2.0,2.0,2.0,6.0,4.0,3.0,6.0,2.0,...,10.0,2.0,2.0,2.0,2.0,5.0,4.0,2.0,5.0,2.0
erelrpe_hh_rel,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
erelrpe_hh_norel,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
erelrpe_os_spouse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
erelrpe_os_partner,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
erelrpe_ss_spouse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
erelrpe_ss_partner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
erelrpe_child,0.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,...,8.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0


# Convert Families

If the HH reference person has an unmarried partner who has children, merge them into the same family

However, if they don't have kids, they are not considered a family. 


In [75]:
from itertools import islice
rows = []

def parse_hh(df):
    """Return a row of data from a household dataframe"""

    pn = df.set_index('pnum')
    
    refp = df[df.erelrpe.isin([1,2])].pnum.iloc[0]
    
    partnum = df[df.erelrpe.isin([3,4,5,6])].pnum.fillna(0)
    partnum = int(partnum.iloc[0]) if len(partnum) > 0 else 0
    
    nfam1 = df[~df.erelrpe.isin([16,17,18])].rfamref.nunique()
    if partnum!=0 and pn.loc[partnum].rfamref != pn.loc[refp].rfamref:
        # Reset the family numbers so that partners and their children are all in the same family
        df.loc[df.rfamref ==  pn.loc[partnum].rfamref ] = pn.loc[refp].rfamref
        rfamkind = np.max([ pn.loc[refp].rfamkind,  pn.loc[partnum].rfamkind])
    else:
        rfamkind = pn.loc[refp].rfamkind
    
    rfamkind =   rfamkind.astype(int)
    
    # Re-compute the number of families, excluding roomates and boarders
    nfam2 = df[~df.erelrpe.isin([16,17,18])].rfamref.nunique()
    
    
    return {
        'pnum': int(df[df.erelrpe.isin([1,2])].pnum), # Person number of the householder
        'partnum': partnum, # person number of the partner
        'nfam1': nfam1, # Number of families in the household, ignoring conversion
        'nfam2': nfam2, # number of familes includeing conversion
        'pfamsize': df[df.rfamref == pn.loc[refp].rfamref].shape[0], # # size of the primary family
        'rfamkind': rfamkind,
        'nresidence': df.eresidenceid.nunique()
        #'tftotinc':, # sum of family member income
        
    }
    
hhg = dfa[dfa.monthcode==1].groupby(['ssuid'])
rows = [ dict( [('ssuid',ssuid)] + list(parse_hh(g).items()) )  for ssuid, g in  islice(hhg,1000)]
   
hh = pd.DataFrame(rows)
hh

,ssuid,pnum,partnum,nfam1,nfam2,pfamsize,rfamkind,nresidence
0,11413607018,101,0,1,1,1,0,1
1,11413613418,101,0,1,1,1,0,1
2,11413646518,102,103,1,1,4,1,1
3,11428574618,101,102,1,1,2,1,1
4,11428577018,101,102,1,1,2,1,1
...,...,...,...,...,...,...,...,...
995,3828555506718,101,102,1,1,4,1,1
996,3828555514918,101,0,1,1,2,2,1
997,3828555518018,101,102,1,1,2,1,1
998,3828555519718,101,0,1,1,2,3,1


In [76]:
hh[hh.nresidence > 1]

,ssuid,pnum,partnum,nfam1,nfam2,pfamsize,rfamkind,nresidence
23,28509223518,102,101,2,1,4,3,2
35,28533413818,101,102,2,1,2,0,2
48,40618528318,101,102,2,1,2,0,2
49,40618531618,101,102,2,1,6,3,2
50,40618594618,101,102,2,1,4,2,2
...,...,...,...,...,...,...,...,...
950,3828503775118,104,101,4,3,2,0,4
959,3828504967618,101,102,2,1,2,0,2
960,3828504968918,101,102,2,1,2,0,2
964,3828505720718,101,102,2,1,2,0,2


In [12]:
df.sort_values(['ssuid','rfamref'])[['pnum', 'ssuid','rfamref', 'tftotinc', 'tptotinc']].head()

,pnum,ssuid,rfamref,tftotinc,tptotinc
11,101,11413607018,101,1738,1738.0
23,101,11413613418,101,2972,2972.0
35,101,11413646518,102,8114,1864.0
47,102,11413646518,102,8114,0.0
59,103,11413646518,102,8114,6250.0


In [46]:
# sum(tptotinc) does equal tftotinc (family income), when grouping by rfamref
t = df.groupby(['ssuid','eresidenceid', 'rfamref']).agg({'tptotinc':'sum', 'tftotinc':'first'})
t[t.tptotinc != t.tftotinc]

,,,tptotinc,tftotinc
ssuid,eresidenceid,rfamref,,


In [47]:
# When grouping by household, sum(tptotinc) equals thtotinc (household income), in 99% of households
# But there are some households, by ssuid,  that have multiple thtotinc values. 
# ssuid == 88186086108518 is a particulr weird case
t = df.groupby(['ssuid', 'eresidenceid']).agg({'tptotinc':'sum', 'thtotinc':'first', 'tftotinc': 'first'})
t[t.tptotinc != t.thtotinc][['tptotinc','thtotinc']]

,,tptotinc,thtotinc
ssuid,eresidenceid,,


In [45]:
'ERESIDENCEID'.lower()

'eresidenceid'

In [78]:
# Here is an example of a households 
dfa[(dfa.ssuid == 3828503775118) & (dfa.monthcode==1)][['pnum', 'ERESIDENCEID'\
    .lower(), 'erel_cat', 'rfamref', 'rfamkind', 'epnpar1','epnpar2','tptotinc','thtotinc']]\
    .sort_values('rfamref')

,pnum,eresidenceid,erel_cat,rfamref,rfamkind,epnpar1,epnpar2,tptotinc,thtotinc
26450,101,100002,os_partner,101,0.0,NaN,NaN,0.0,0
26462,102,100003,child,102,0.0,104.0,NaN,NaN,0
26474,103,100002,child,103,0.0,104.0,NaN,NaN,0
26486,104,100001,hh_rel,104,0.0,NaN,NaN,596.0,596


In [66]:
# Here is an example of a households 
dfa[(dfa.ssuid == 88186086108518) & (dfa.monthcode==12)][['pnum', 'ERESIDENCEID'\
    .lower(), 'erel_cat', 'rfamref', 'rfamkind', 'epnpar1','epnpar2','tptotinc','thtotinc']]\
    .sort_values('rfamref')

,pnum,eresidenceid,erel_cat,rfamref,rfamkind,epnpar1,epnpar2,tptotinc,thtotinc
746358,101,100001,hh_rel,101,2.0,NaN,NaN,3609.0,12753
746370,102,100001,child,101,2.0,101.0,NaN,2607.0,12753
746394,104,100001,roommate,104,2.0,NaN,NaN,2.0,12753
746406,105,100001,roommate,104,2.0,104.0,NaN,NaN,12753
746382,103,100001,roommate,106,1.0,106.0,111.0,2899.0,12753
746418,106,100001,roommate,106,1.0,NaN,NaN,3636.0,12753
746430,107,100001,roommate,106,1.0,106.0,111.0,0.0,12753
746478,111,100001,roommate,106,1.0,NaN,NaN,0.0,12753
746442,108,100002,roommate,108,0.0,NaN,NaN,1110.0,1110
746454,109,100002,roommate,109,0.0,NaN,NaN,NaN,1110


In [70]:
dfa[(dfa.eresidenceid == 100002) & (dfa.monthcode==12)][['ssuid', 'pnum',
    'erel_cat', 'rfamref', 'rfamkind', 'epnpar1','epnpar2','tptotinc','thtotinc']]\
    .sort_values('rfamref').head(50)

,ssuid,pnum,erel_cat,rfamref,rfamkind,epnpar1,epnpar2,tptotinc,thtotinc
143,11455225318,101,hh_norel,101,0.0,NaN,NaN,1376.0,1376
420432,66755513631618,103,child,101,2.0,101.0,102.0,0.0,3303
420408,66755513631618,101,hh_rel,101,2.0,NaN,NaN,220.0,3303
420396,66755513629118,101,hh_norel,101,0.0,NaN,NaN,1521.0,1521
419632,66755220906018,103,grandchild,101,2.0,102.0,NaN,NaN,13579
419620,66755220906018,102,child,101,2.0,101.0,NaN,3.0,13579
419608,66755220906018,101,hh_rel,101,2.0,NaN,NaN,13576.0,13579
418773,66749404990518,101,hh_norel,101,0.0,NaN,NaN,1360.0,1360
417482,66741896716018,104,child,101,1.0,101.0,102.0,NaN,11503
417470,66741896716018,103,child,101,1.0,101.0,102.0,NaN,11503


In [77]:
# Here is an example of a households 
dfa[(dfa.ssuid == 28509223518) & (dfa.monthcode==12)][['pnum', 'ERESIDENCEID'\
    .lower(), 'erel_cat', 'rfamref', 'rfamkind', 'epnpar1','epnpar2','tptotinc','thtotinc']]\
    .sort_values('rfamref')

,pnum,eresidenceid,erel_cat,rfamref,rfamkind,epnpar1,epnpar2,tptotinc,thtotinc
515,101,100001,os_partner,101,0.0,NaN,NaN,0.0,5390
527,102,100001,hh_rel,102,3.0,NaN,NaN,5390.0,5390
539,103,100001,child,102,3.0,102.0,NaN,NaN,5390
551,104,100001,child,102,3.0,102.0,NaN,NaN,5390


In [ ]:
for ssuid in hh[hh.nfam2 >2].ssuid:
    print(df[df.ssuid == ssuid][['pnum', 'erel_cat', 'rfamref', 'rfamkind', 'epnpar1','epnpar2']]) #.value_counts()

In [ ]:
df.loc[10632].tage

In [48]:
df.eresidenceid.unique()

array([100001, 100002, 100003, 100004, 100006, 100005, 100007])